<a href="https://colab.research.google.com/github/dmeoli/NeuroSAT/blob/master/GATQSAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd gdrive/My Drive

/content/gdrive/My Drive


In [3]:
import os

if not os.path.isdir('neuroSAT'):
  !git clone --recurse-submodules https://github.com/dmeoli/neuroSAT
  %cd neuroSAT
else:
  %cd neuroSAT
  !git pull

/content/gdrive/My Drive/neuroSAT
Already up to date.


In [4]:
!pip install -r requirements.txt

Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
     |████████████████████████████████| 124 kB 4.2 MB/s 
     |████████████████████████████████| 325 kB 30.2 MB/s 
     |████████████████████████████████| 3.7 MB 4.0 MB/s 
     |████████████████████████████████| 10.4 MB 61.7 MB/s 
     |████████████████████████████████| 407 kB 73.2 MB/s 
     |████████████████████████████████| 45 kB 4.6 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.2-py3-none-any.whl size=535570 sha256=2acda3f70e8e7f5ba7aa4be83edd85639203060bc6e2b9e0cf9aec105a6c9c77
  Stored in directory: /root/.cache/pip/wheels/3f/08/13/2321517088bb2e95bfd0e45033bb9c923189e5b2078e0be4ef
Successfully built torch-geometric


In [5]:
datasets = {'uniform-random-3-sat': {'train': ['uf50-218', 'uuf50-218',
                                               'uf100-430', 'uuf100-430'],
                                     'val': ['uf50-218', 'uuf50-218',
                                             'uf100-430', 'uuf100-430'],
                                     'inner_test': ['uf50-218', 'uuf50-218',
                                                    'uf100-430', 'uuf100-430'],
                                     'test': ['uf250-1065', 'uuf250-1065']},
            'graph-coloring': {'train': ['flat50-115'],
                               'val': ['flat75-180'],
                               'test': ['flat30-60',
                                        'flat100-239',
                                        'flat125-301',
                                        'flat150-360',
                                        'flat175-417',
                                        'flat200-479']}}

# GATQSAT

In [6]:
%cd GraphQSAT

/content/gdrive/My Drive/neuroSAT/GraphQSAT


### Build C++

In [7]:
%cd minisat
!sudo ln -s --force /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy /usr/include/numpy  # https://stackoverflow.com/a/44935933/5555994
!make distclean && CXXFLAGS=-w make && make python-wrap PYTHON=python3.7
!apt install swig
!swig -fastdispatch -c++ -python minisat/gym/GymSolver.i
%cd ..

/content/gdrive/My Drive/neuroSAT/GraphQSAT/minisat
rm -f   build/release/minisat/core/Solver.o  build/release/minisat/core/Main.o  build/release/minisat/simp/SimpSolver.o  build/release/minisat/simp/Main.o  build/release/minisat/utils/System.o  build/release/minisat/utils/Options.o  build/release/minisat/gym/GymSolver.o   build/debug/minisat/core/Solver.o  build/debug/minisat/core/Main.o  build/debug/minisat/simp/SimpSolver.o  build/debug/minisat/simp/Main.o  build/debug/minisat/utils/System.o  build/debug/minisat/utils/Options.o  build/debug/minisat/gym/GymSolver.o   build/profile/minisat/core/Solver.o  build/profile/minisat/core/Main.o  build/profile/minisat/simp/SimpSolver.o  build/profile/minisat/simp/Main.o  build/profile/minisat/utils/System.o  build/profile/minisat/utils/Options.o  build/profile/minisat/gym/GymSolver.o   build/dynamic/minisat/core/Solver.o  build/dynamic/minisat/core/Main.o  build/dynamic/minisat/simp/SimpSolver.o  build/dynamic/minisat/simp/Main.o  build/dynam

## Uniform Random 3-SAT

We split *uf50-218* and *uf100-430* into three subsets: 800 training problems, 100 validation and 100 test problems.
For generalization experiments, we use 100 problems from all the other benchmarks.

In [8]:
PROBLEM_TYPE='uniform-random-3-sat'

In [ ]:
!bash train_val_test_split.sh "$PROBLEM_TYPE"

Copying files: 2100 files [00:14, 144.10 files/s]
Copying files: 1300 files [05:44,  3.77 files/s]


### Add metadata for evaluation (train and validation set)

In [ ]:
for TRAIN_PROBLEM_NAME, VAL_PROBLEM_NAME in zip(datasets[PROBLEM_TYPE]['train'],
                                                datasets[PROBLEM_TYPE]['val']):
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/train/"$TRAIN_PROBLEM_NAME"
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/val/"$VAL_PROBLEM_NAME"

Streaming output truncated to the last 5000 lines.
Rollout 349, steps 91, num_restarts 0.
Rollout 350, steps 33, num_restarts 0.
Rollout 351, steps 17, num_restarts 0.
Rollout 352, steps 27, num_restarts 0.
Rollout 353, steps 16, num_restarts 0.
Rollout 354, steps 22, num_restarts 0.
Rollout 355, steps 44, num_restarts 0.
Rollout 356, steps 26, num_restarts 0.
Rollout 357, steps 9, num_restarts 0.
Rollout 358, steps 29, num_restarts 0.
Rollout 359, steps 46, num_restarts 0.
Rollout 360, steps 16, num_restarts 0.
Rollout 361, steps 52, num_restarts 0.
Rollout 362, steps 70, num_restarts 0.
Rollout 363, steps 30, num_restarts 0.
Rollout 364, steps 35, num_restarts 0.
Rollout 365, steps 17, num_restarts 0.
Rollout 366, steps 14, num_restarts 0.
Rollout 367, steps 65, num_restarts 0.
Rollout 368, steps 27, num_restarts 0.
Rollout 369, steps 25, num_restarts 0.
Rollout 370, steps 56, num_restarts 0.
Rollout 371, steps 32, num_restarts 0.
Rollout 372, steps 38, num_restarts 0.
Rollout 373, s

### Train

In [ ]:
for TRAIN_PROBLEM_NAME, VAL_PROBLEM_NAME in zip(datasets[PROBLEM_TYPE]['train'],
                                                datasets[PROBLEM_TYPE]['val']):
  !python dqn.py \
    --logdir log \
    --env-name sat-v0 \
    --train-problems-paths ../data/"$PROBLEM_TYPE"/train/"$TRAIN_PROBLEM_NAME" \
    --eval-problems-paths ../data/"$PROBLEM_TYPE"/val/"$VAL_PROBLEM_NAME" \
    --lr 0.00002 \
    --bsize 64 \
    --buffer-size 20000 \
    --eps-init 1.0 \
    --eps-final 0.01 \
    --gamma 0.99 \
    --eps-decay-steps 30000 \
    --batch-updates 50000 \
    --history-len 1 \
    --init-exploration-steps 5000 \
    --step-freq 4 \
    --target-update-freq 10 \
    --loss mse \
    --opt adam \
    --save-freq 500 \
    --grad_clip 1 \
    --grad_clip_norm_type 2 \
    --eval-freq 1000 \
    --eval-time-limit 3600 \
    --core-steps 4 \
    --expert-exploration-prob 0.0 \
    --priority_alpha 0.5 \
    --priority_beta 0.5 \
    --e2v-aggregator sum \
    --n_hidden 1 \
    --hidden_size 64 \
    --decoder_v_out_size 32 \
    --decoder_e_out_size 1 \
    --decoder_g_out_size 1 \
    --encoder_v_out_size 32 \
    --encoder_e_out_size 32 \
    --encoder_g_out_size 32 \
    --core_v_out_size 64 \
    --core_e_out_size 64 \
    --core_g_out_size 32 \
    --activation relu \
    --penalty_size 0.1 \
    --train_time_max_decisions_allowed 500 \
    --test_time_max_decisions_allowed 500 \
    --no_max_cap_fill_buffer \
    --lr_scheduler_gamma 1 \
    --lr_scheduler_frequency 3000 \
    --independent_block_layers 0

Namespace(activation='relu', batch_updates=50000, bsize=64, buffer_size=20000, compare_with_restarts=False, core_e_out_size=64, core_g_out_size=32, core_steps=4, core_v_out_size=64, debug=False, decoder_e_out_size=1, decoder_g_out_size=1, decoder_v_out_size=32, device=device(type='cuda'), dump_timings_path=None, e2v_aggregator='sum', encoder_e_out_size=32, encoder_g_out_size=32, encoder_v_out_size=32, env_name='sat-v0', eps_decay_steps=30000, eps_final=0.01, eps_init=1.0, eval_freq=1000, eval_problems_paths='../data/uniform-random-3-sat/val/uuf50-218', eval_separately_on_each=True, eval_time_limit=3600, expert_exploration_prob=0.0, gamma=0.99, grad_clip=1.0, grad_clip_norm_type=2.0, hidden_size=64, history_len=1, independent_block_layers=0, init_exploration_steps=5000, logdir='runs/Sep12_10-11-09_4730bf8f8ad0', loss='mse', lr=2e-05, lr_scheduler_frequency=3000, lr_scheduler_gamma=1.0, max_cap_fill_buffer=False, model_checkpoint=None, model_dir=None, n_hidden=1, no_cuda=False, opt='adam

### Add metadata for evaluation (test set)

In [ ]:
for PROBLEM_NAME in datasets[PROBLEM_TYPE]['inner_test']:
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/test/"$PROBLEM_NAME"

for PROBLEM_NAME in datasets[PROBLEM_TYPE]['test']:
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$PROBLEM_NAME"

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/neuro-sat/data/uniform-random-3-sat/test/uf50-218/METADATA'
No metadata available, that is fine for metadata generator.
Testing agent <MiniSAT Agent>... with_restarts is set to False
Rollout 1, steps 38, num_restarts 0.
Rollout 2, steps 16, num_restarts 0.
Rollout 3, steps 8, num_restarts 0.
Rollout 4, steps 15, num_restarts 0.
Rollout 5, steps 87, num_restarts 0.
Rollout 6, steps 17, num_restarts 0.
Rollout 7, steps 13, num_restarts 0.
Rollout 8, steps 27, num_restarts 0.
Rollout 9, steps 29, num_restarts 0.
Rollout 10, steps 28, num_restarts 0.
Rollout 11, steps 17, num_restarts 0.
Rollout 12, steps 40, num_restarts 0.
Rollout 13, steps 67, num_restarts 0.
Rollout 14, steps 17, num_restarts 0.
Rollout 15, steps 66, num_restarts 0.
Rollout 16, steps 62, num_restarts 0.
Rollout 17, steps 38, num_restarts 0.
Rollout 18, steps 16, num_restarts 0.
Rollout 19, steps 32, num_restarts 0.
Rollout 20, steps 94, num_restarts 0.
Rollo

### Evaluate

In [ ]:
models = {'uf50-218': ('Aug04_16-28-54_98b46997a24d', 
                       'model_50001'),
          'uuf50-218': ('Sep12_10-11-09_4730bf8f8ad0', 
                        'model_50006'),
          'uf100-430': ('Aug04_21-40-43_98b46997a24d', 
                        'model_50061'),
          'uuf100-430': ('Sep12_16-06-06_4730bf8f8ad0', 
                         'model_45085')}

We test these trained models on the inner test sets.

In [ ]:
for PROBLEM_NAME in datasets[PROBLEM_TYPE]['inner_test']:
  for MODEL_DECISION in [0, 10, 50, 100, 300, 500, 1000]:
    MODEL_DIR = models[PROBLEM_NAME][0]
    CHECKPOINT = models[PROBLEM_NAME][1]
    !python evaluate.py \
      --logdir log \
      --env-name sat-v0 \
      --core-steps -1 \
      --eps-final 0.0 \
      --eval-time-limit 100000000000000 \
      --no_restarts \
      --test_time_max_decisions_allowed "$MODEL_DECISION" \
      --eval-problems-paths ../data/"$PROBLEM_TYPE"/test/"$PROBLEM_NAME" \
      --model-dir runs/"$MODEL_DIR" \
      --model-checkpoint "$CHECKPOINT".chkp \
      >> runs/"$MODEL_DIR"/"$PROBLEM_NAME"-gatqsat-max"$MODEL_DECISION".tsv

We test the trained models on the outer test sets.

In [ ]:
for SAT_MODEL in models.keys():
  for PROBLEM_NAME in datasets[PROBLEM_TYPE]['test']:
    if ((SAT_MODEL.startswith('uf') and PROBLEM_NAME.startswith('uf')) or
        (SAT_MODEL.startswith('uuf') and PROBLEM_NAME.startswith('uuf'))):
      for MODEL_DECISION in [0, 10, 50, 100, 300, 500, 1000]:
        MODEL_DIR = models[SAT_MODEL][0]
        CHECKPOINT = models[SAT_MODEL][1]
        !python evaluate.py \
          --logdir log \
          --env-name sat-v0 \
          --core-steps -1 \
          --eps-final 0.0 \
          --eval-time-limit 100000000000000 \
          --no_restarts \
          --test_time_max_decisions_allowed "$MODEL_DECISION" \
          --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$PROBLEM_NAME" \
          --model-dir runs/"$MODEL_DIR" \
          --model-checkpoint "$CHECKPOINT".chkp \
          >> runs/"$MODEL_DIR"/"$PROBLEM_NAME"-gqsat-max"$MODEL_DECISION".tsv

In [ ]:
%cd ..

/content/gdrive/My Drive/neuro-sat


## Graph Coloring

Graph coloring benchmarks have only 100 problems each, except for *flat50-115* which contains 1000, so we do not split them into training/validation/test sets using *flat50-115* for training and *flat75-180* to do model selection.

In [9]:
PROBLEM_TYPE='graph-coloring'

### Add metadata for evaluation (train and validation set)

In [ ]:
for PROBLEM_NAME in datasets[PROBLEM_TYPE]['train'] + datasets[PROBLEM_TYPE]['val']:
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$PROBLEM_NAME"

[Errno 2] No such file or directory: '/content/gdrive/My Drive/neuro-sat/data/graph-coloring/flat50-115/METADATA'
No metadata available, that is fine for metadata generator.
Testing agent <MiniSAT Agent>... with_restarts is set to False
Rollout 1, steps 60, num_restarts 0.
Rollout 2, steps 13, num_restarts 0.
Rollout 3, steps 15, num_restarts 0.
Rollout 4, steps 10, num_restarts 0.
Rollout 5, steps 19, num_restarts 0.
Rollout 6, steps 18, num_restarts 0.
Rollout 7, steps 14, num_restarts 0.
Rollout 8, steps 7, num_restarts 0.
Rollout 9, steps 18, num_restarts 0.
Rollout 10, steps 12, num_restarts 0.
Rollout 11, steps 13, num_restarts 0.
Rollout 12, steps 18, num_restarts 0.
Rollout 13, steps 23, num_restarts 0.
Rollout 14, steps 31, num_restarts 0.
Rollout 15, steps 28, num_restarts 0.
Rollout 16, steps 26, num_restarts 0.
Rollout 17, steps 12, num_restarts 0.
Rollout 18, steps 13, num_restarts 0.
Rollout 19, steps 18, num_restarts 0.
Rollout 20, steps 17, num_restarts 0.
Rollout 21, s

### Train

In [ ]:
for TRAIN_PROBLEM_NAME, VAL_PROBLEM_NAME in zip(datasets[PROBLEM_TYPE]['train'],
                                                datasets[PROBLEM_TYPE]['val']):
  !python dqn.py \
    --logdir log \
    --env-name sat-v0 \
    --train-problems-paths ../data/"$PROBLEM_TYPE"/"$TRAIN_PROBLEM_NAME" \
    --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$VAL_PROBLEM_NAME" \
    --lr 0.00002 \
    --bsize 64 \
    --buffer-size 20000 \
    --eps-init 1.0 \
    --eps-final 0.01 \
    --gamma 0.99 \
    --eps-decay-steps 30000 \
    --batch-updates 50000 \
    --history-len 1 \
    --init-exploration-steps 5000 \
    --step-freq 4 \
    --target-update-freq 10 \
    --loss mse \
    --opt adam \
    --save-freq 500 \
    --grad_clip 0.1 \
    --grad_clip_norm_type 2 \
    --eval-freq 1000 \
    --eval-time-limit 3600 \
    --core-steps 4 \
    --expert-exploration-prob 0.0 \
    --priority_alpha 0.5 \
    --priority_beta 0.5 \
    --e2v-aggregator sum \
    --n_hidden 1 \
    --hidden_size 64 \
    --decoder_v_out_size 32 \
    --decoder_e_out_size 1 \
    --decoder_g_out_size 1 \
    --encoder_v_out_size 32 \
    --encoder_e_out_size 32 \
    --encoder_g_out_size 32 \
    --core_v_out_size 64 \
    --core_e_out_size 64 \
    --core_g_out_size 32 \
    --activation relu \
    --penalty_size 0.1 \
    --train_time_max_decisions_allowed 500 \
    --test_time_max_decisions_allowed 500 \
    --no_max_cap_fill_buffer \
    --lr_scheduler_gamma 1 \
    --lr_scheduler_frequency 3000 \
    --independent_block_layers 0

EncodeProcessDecode(
  (encoder): IndependentGraphNet(
    (op): ModifiedMetaLayer(
        edge_model=EdgeModel(
      (edge_mlp): Sequential(
        (0): Linear(in_features=2, out_features=32, bias=True)
        (1): ReLU()
        (2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      )
    ),
        node_model=NodeModel(
      (node_mlp): Sequential(
        (0): Linear(in_features=2, out_features=32, bias=True)
        (1): ReLU()
        (2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      )
    ),
        global_model=GlobalModel(
      (global_mlp): Sequential(
        (0): Linear(in_features=1, out_features=32, bias=True)
        (1): ReLU()
        (2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      )
    )
    )
  )
  (core): GraphNet(
    (attention_model): Sequential(
      (0): EdgeGATConv(64, 64, heads=3)
      (1): ReLU(inplace=True)
      (2): EdgeGATConv(192, 64, heads=1)
      (3): ReLU(inplace=True)
      (4): LayerNorm((64,), eps=1

### Add metadata for evaluation (test set)

In [ ]:
for PROBLEM_NAME in datasets[PROBLEM_TYPE]['test']:
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$PROBLEM_NAME"

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/neuro-sat/data/uniform-random-3-sat/test/uf50-218/METADATA'
No metadata available, that is fine for metadata generator.
Testing agent <MiniSAT Agent>... with_restarts is set to False
Rollout 1, steps 38, num_restarts 0.
Rollout 2, steps 16, num_restarts 0.
Rollout 3, steps 8, num_restarts 0.
Rollout 4, steps 15, num_restarts 0.
Rollout 5, steps 87, num_restarts 0.
Rollout 6, steps 17, num_restarts 0.
Rollout 7, steps 13, num_restarts 0.
Rollout 8, steps 27, num_restarts 0.
Rollout 9, steps 29, num_restarts 0.
Rollout 10, steps 28, num_restarts 0.
Rollout 11, steps 17, num_restarts 0.
Rollout 12, steps 40, num_restarts 0.
Rollout 13, steps 67, num_restarts 0.
Rollout 14, steps 17, num_restarts 0.
Rollout 15, steps 66, num_restarts 0.
Rollout 16, steps 62, num_restarts 0.
Rollout 17, steps 38, num_restarts 0.
Rollout 18, steps 16, num_restarts 0.
Rollout 19, steps 32, num_restarts 0.
Rollout 20, steps 94, num_restarts 0.
Rollo

### Evaluate

In [ ]:
MODEL_DIR='Sep10_12-34-08_19c1d7476815'
CHECKPOINT='model_50003'

In [ ]:
for PROBLEM_NAME in datasets[PROBLEM_TYPE]['test']:
  for MODEL_DECISION in [0, 10, 50, 100, 300, 500, 1000]:
    !python evaluate.py \
      --logdir log \
      --env-name sat-v0 \
      --core-steps -1 \
      --eps-final 0.0 \
      --eval-time-limit 100000000000000 \
      --no_restarts \
      --test_time_max_decisions_allowed "$MODEL_DECISION" \
      --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$PROBLEM_NAME" \
      --model-dir runs/"$MODEL_DIR" \
      --model-checkpoint "$CHECKPOINT".chkp \
      >> runs/"$MODEL_DIR"/"$PROBLEM_NAME"-gatqsat-max"$MODEL_DECISION".tsv

In [ ]:
%cd ..

/content/gdrive/My Drive/neuro-sat
